# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = "plagiarism_data"

# set prefix, a descriptive name for a directory  
prefix = "features"

# upload all data to S3
input_data = sagemaker_session.upload_data(data_dir, bucket, prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [93]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

features/linear-learner-2020-03-05-21-37-44-072/output/model.tar.gz
features/test.csv
features/train.csv
sagemaker-pytorch-2020-02-17-16-28-36-082/source/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-16-37-01-899/source/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-16-37-55-986/debug-output/training_job_end.ts
sagemaker-pytorch-2020-02-17-16-37-55-986/output/model.tar.gz
sagemaker-pytorch-2020-02-17-16-37-55-986/source/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-18-34-30-345/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-20-12-47-246/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-20-26-43-512/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-20-49-39-285/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-21-02-20-733/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-21-44-16-567/debug-output/training_job_end.ts
sagemaker-pytorch-2020-02-17-21-44-16-567/output/model.tar.gz
sagemaker-pytorch-2020-02-17-21-44-16-567/source/sourcedir.tar.gz
sagemaker-pytorch-2020-02-17-22-24-40-823/sourcedir.tar.gz
sag

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [4]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [8]:
import os

train_data = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None)
train_data.head(5)

,0,1,2,3,4,5
0,0,0.398148,0.079070,0.009346,0.000000,0.191781
1,1,0.869369,0.719457,0.613636,0.515982,0.846491
2,1,0.593583,0.268817,0.156757,0.108696,0.316062
3,0,0.544503,0.115789,0.031746,0.005319,0.242574
4,0,0.329502,0.053846,0.007722,0.003876,0.161172


In [9]:
from sagemaker import LinearLearner

# specify an output path
result_prefix = "result"
output_path = 's3://{}/{}'.format(bucket, result_prefix)

# Labels are in the first column
train_y = train_data.iloc[:,0].values
train_x = train_data.iloc[:,1:].values
    



In [100]:
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [16]:
%%time

# Train your estimator on S3 training data

train_x_np = train_x.astype('float32')
train_y_np = train_y.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

linear.fit(formatted_train_data)

2020-03-05 21:37:44 Starting - Starting the training job...
2020-03-05 21:37:45 Starting - Launching requested ML instances......
2020-03-05 21:38:50 Starting - Preparing the instances for training......
2020-03-05 21:40:09 Downloading - Downloading input data...
2020-03-05 21:40:42 Training - Training image download completed. Training in progress.
2020-03-05 21:40:42 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
[03/05/2020 21:40:39 INFO 140509190592320] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step':


2020-03-05 21:40:50 Completed - Training job completed
Training seconds: 41
Billable seconds: 41
CPU times: user 527 ms, sys: 38.9 ms, total: 566 ms
Wall time: 3min 42s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [19]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


-----------------!CPU times: user 285 ms, sys: 6.18 ms, total: 292 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [22]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x.values.astype('float32'))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [60]:
np.array(pred_y)

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 0., 0.])

In [18]:
import numpy as np

def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    test_y_preds = predictor.predict(test_x.values.astype('float32'))
    if not isinstance(test_y_preds, np.ndarray):
        test_preds = []
        for pred in test_y_preds:
            test_preds.append(pred.label['predicted_label'].float32_tensor.values[0])
        test_preds = np.array(test_preds)
    else:
        # from PyTorch model
        test_preds = test_y_preds.reshape(1,-1)[0] # reshape the data into array
        test_preds = np.around(test_preds)
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [19]:
# Second: calculate the test accuracy
metrics = evaluate(predictor, 
                   test_x.values.astype('float32'), 
                   test_y.values.astype('float32'), 
                   verbose=True)

accuracy = metrics['Accuracy']

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

prediction (col)  0.0  1.0
actual (row)              
0.0                10    0
1.0                 0   15

Recall:     1.000
Precision:  1.000
Accuracy:   1.000

1.0

Predicted class labels: 
[[0.99989974]
 [0.99801636]
 [0.970379  ]
 [0.6282166 ]
 [0.9969133 ]
 [0.9999324 ]
 [0.31661564]
 [0.22658157]
 [0.3161289 ]
 [0.3056584 ]
 [0.32871035]
 [0.4053763 ]
 [0.55255497]
 [0.999572  ]
 [0.9999218 ]
 [0.83338374]
 [0.9679785 ]
 [0.9999349 ]
 [0.24331246]
 [0.9999124 ]
 [0.3107227 ]
 [0.9998536 ]
 [0.99964345]
 [0.24464606]
 [0.40003422]]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [21]:
# Double check results

pred_y = []
for pred in test_y_preds:
    pred_y.append(pred.label['predicted_label'].float32_tensor.values[0])
        
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, pred_y)

array([[10,  0],
       [ 0, 15]])

In [22]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, pred_y)
print(accuracy)

1.0


In [129]:
# Cleanup and attempting PyTorch model for educational purposes
predictor.delete_endpoint()

# Exercise: Train PyTorch estimator and compare results with first model

In [10]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="source_pytorch",
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.m5.4xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': len(train_x[0]), # 5 features
                        'epochs': 100,
                        'hidden_dim': 30,
                        'output_dim': 1
                    })

estimator.fit({'train': input_data})

2020-03-06 09:27:20 Starting - Starting the training job...
2020-03-06 09:27:21 Starting - Launching requested ML instances......
2020-03-06 09:28:27 Starting - Preparing the instances for training...
2020-03-06 09:29:11 Downloading - Downloading input data...
2020-03-06 09:29:39 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-06 09:29:40,216 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-06 09:29:40,219 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-06 09:29:40,230 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-06 09:29:43,250 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-06 09:29:43,445 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py


2020-03-06 09:30:05 Uploading - Uploading generated training model
2020-03-06 09:30:05 Completed - Training job completed
Training seconds: 54
Billable seconds: 54


# Evaluate PyTorch model

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [12]:
%%time
# deploy the NN model
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------!CPU times: user 212 ms, sys: 14.6 ms, total: 226 ms
Wall time: 5min 31s


In [17]:
test_y_preds = predictor.predict(test_x.values.astype('float32'))

assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

metrics = evaluate(predictor, 
                   test_x.values.astype('float32'), 
                   test_y.values.astype('float32'), 
                   verbose=True)

accuracy = metrics['Accuracy']

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

Test passed!
prediction (col)  0.226582  0.243312  0.244646  0.305658  0.310723  0.316129  \
actual (row)                                                                   
0.0                      1         1         1         1         1         1   
1.0                      0         0         0         0         0         0   

prediction (col)  0.316616  0.328710  0.400034  0.405376  ...  0.996913  \
actual (row)                                              ...             
0.0                      1         1         1         1  ...         0   
1.0                      0         0         0         0  ...         1   

prediction (col)  0.998016  0.999572  0.999643  0.999854  0.999900  0.999912  \
actual (row)                                                                   
0.0                      0         0         0         0         0         0   
1.0                      1         1         1         1         1         1   

prediction (col)  0.999922  0.999932  0.9999

In [23]:
# Double check the results
test_preds = test_y_preds.reshape(1,-1)[0] # reshape the data into array
pred_y = np.around(test_preds)

confusion_matrix(test_y, pred_y)

array([[10,  0],
       [ 0, 15]])

In [24]:
accuracy = accuracy_score(test_y, pred_y)
print(accuracy)

1.0


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: A simple linear model returned no false positives or false negatives. I trained the model for large number of epochs (both simple Linear model and PyTorch model), and the dataset size is quite small, so this behaviour is expected. Also we provided 5 feautes for model to be trained on. Considering all this, our model learned well enough.




### Question 2: How did you decide on the type of model to use? 

** Answer**: Dataset if a perfect fit for classic binary classification problem, so using LineaLearner algorithm should already give us nice results that further can be improved with models like Random Forest or simple Artificial Neural Network.
As expected, LinearLearner provided us with 100% accuraccy and precision. Without any false negatives and false positives.
For an excercise I also constructed PyTorch model with one hidden dimention and trained it for 100 epochs. It also achieved 100% accuracy. this PyTorch model might have better insight on underlying data, and might be perform slighly better on previously unseen data.


----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [27]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [28]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'BA01EB43EC64907C',
   'HostId': 'qlPdY/IyAuLrUB49z2jza1dJ4HinOnZZzKZpAItN43e+9jLMYF4F7/igHVE+LoAaI7PbC4BWGHo=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'qlPdY/IyAuLrUB49z2jza1dJ4HinOnZZzKZpAItN43e+9jLMYF4F7/igHVE+LoAaI7PbC4BWGHo=',
    'x-amz-request-id': 'BA01EB43EC64907C',
    'date': 'Fri, 06 Mar 2020 10:13:18 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-pytorch-2020-02-17-20-12-47-246/sourcedir.tar.gz'},
   {'Key': 'result/sagemaker-pytorch-2020-03-06-09-27-19-666/output/model.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-03-05-23-33-49-368/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-03-05-21-37-03-074/matrix_0.pbr'},
   {'Key': 'sagemaker-pytorch-2020-03-05-23-41-24-723/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-03-06-00-08-11-2

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!